# 참고
사실상 본 글은 다음 사이트의 내용 및 코드를 보고 따라한 것에 불과한 글임
https://github.com/MLStudy24freecafe/TaeHoonJin/blob/master/Crime(XGB%2CLGBM).ipynb

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import os as os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import scipy as sp
from scipy.special import boxcox1p
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from sklearn.impute import SimpleImputer
plt.style.use('seaborn')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Crime/train.csv',parse_dates=['Dates'])
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Crime/test.csv',parse_dates=['Dates'],index_col='Id')
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Crime/sampleSubmission.csv')

In [3]:
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [4]:
test

,Dates,DayOfWeek,PdDistrict,Address,X,Y
Id,,,,,,
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
...,...,...,...,...,...,...
884257,2003-01-01 00:01:00,Wednesday,MISSION,2600 Block of BRYANT ST,-122.408983,37.751987
884258,2003-01-01 00:01:00,Wednesday,NORTHERN,1900 Block of WASHINGTON ST,-122.425342,37.792681
884259,2003-01-01 00:01:00,Wednesday,INGLESIDE,5500 Block of MISSION ST,-122.445418,37.712075


In [0]:
test_id=test.index

In [0]:
test_id=pd.DataFrame(test_id)

In [7]:
train.duplicated().sum()

2323

In [8]:
train.drop_duplicates(inplace=True)
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [0]:
outlier=[660485, 660711, 660712, 661106, 666430, 667042, 669946, 671709,
            673078, 673114, 673115, 673510, 674043, 675070, 675073, 675288,
            675540, 677036, 677260, 677268, 677828, 678326, 678539, 678540,
            679643, 682780, 683862, 684406, 685719, 685720, 686997, 687519,
            687527, 688950, 692258, 712309, 719993, 727080, 754828, 756059,
            759292, 759397, 767318, 767501, 767502, 772415, 772431, 773159,
            783919, 788574, 788575, 789255, 798834, 823378, 833362, 833921,
            839585, 842259, 842261, 842434, 842435, 842828, 844995, 845842,
            852880, 857248, 871198]

In [10]:
len(outlier)

67

In [11]:
train=train.drop(index=outlier)
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [0]:
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dates'].dt.date,  errors='coerce')
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    
    data.drop(columns=['Dates','Date','Address'], inplace=True)
        
    return data

In [13]:
train

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


In [14]:
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)
train

,Category,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
0,WARRANTS,2,NORTHERN,-122.425892,37.774599,4510,13,5,2015,23,53,False
1,OTHER OFFENSES,2,NORTHERN,-122.425892,37.774599,4510,13,5,2015,23,53,False
2,OTHER OFFENSES,2,NORTHERN,-122.424363,37.800414,4510,13,5,2015,23,33,False
3,LARCENY/THEFT,2,NORTHERN,-122.426995,37.800873,4510,13,5,2015,23,30,True
4,LARCENY/THEFT,2,PARK,-122.438738,37.771541,4510,13,5,2015,23,30,True
...,...,...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,0,TARAVAL,-122.459033,37.714056,0,6,1,2003,0,15,False
878045,LARCENY/THEFT,0,INGLESIDE,-122.447364,37.731948,0,6,1,2003,0,1,True
878046,LARCENY/THEFT,0,SOUTHERN,-122.403390,37.780266,0,6,1,2003,0,1,False
878047,VANDALISM,0,SOUTHERN,-122.390531,37.780607,0,6,1,2003,0,1,False


In [15]:
test

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
Id,,,,,,,,,,,
0,6,BAYVIEW,-122.399588,37.735051,4512,10,5,2015,23,59,True
1,6,BAYVIEW,-122.391523,37.732432,4512,10,5,2015,23,51,False
2,6,NORTHERN,-122.426002,37.792212,4512,10,5,2015,23,50,True
3,6,INGLESIDE,-122.437394,37.721412,4512,10,5,2015,23,45,True
4,6,INGLESIDE,-122.437394,37.721412,4512,10,5,2015,23,45,True
...,...,...,...,...,...,...,...,...,...,...,...
884257,2,MISSION,-122.408983,37.751987,0,1,1,2003,0,1,True
884258,2,NORTHERN,-122.425342,37.792681,0,1,1,2003,0,1,True
884259,2,INGLESIDE,-122.445418,37.712075,0,1,1,2003,0,1,True


In [16]:
train['PdDistrict'].value_counts()

SOUTHERN      156520
MISSION       119722
NORTHERN      105093
BAYVIEW        89089
CENTRAL        85262
TENDERLOIN     81627
INGLESIDE      78718
TARAVAL        65396
PARK           49159
RICHMOND       45073
Name: PdDistrict, dtype: int64

In [0]:
#PdDistrict , Category 변수를 LableEncoding

le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])

In [0]:
test['PdDistrict'] = le1.transform(test['PdDistrict'])

In [0]:
le2 = LabelEncoder()
train_x = train.drop(columns=['Category'])
train_y= le2.fit_transform(train['Category'])

In [20]:
train_x

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
0,2,4,-122.425892,37.774599,4510,13,5,2015,23,53,False
1,2,4,-122.425892,37.774599,4510,13,5,2015,23,53,False
2,2,4,-122.424363,37.800414,4510,13,5,2015,23,33,False
3,2,4,-122.426995,37.800873,4510,13,5,2015,23,30,True
4,2,5,-122.438738,37.771541,4510,13,5,2015,23,30,True
...,...,...,...,...,...,...,...,...,...,...,...
878044,0,8,-122.459033,37.714056,0,6,1,2003,0,15,False
878045,0,2,-122.447364,37.731948,0,6,1,2003,0,1,True
878046,0,7,-122.403390,37.780266,0,6,1,2003,0,1,False
878047,0,7,-122.390531,37.780607,0,6,1,2003,0,1,False


In [21]:
train_y_d=pd.DataFrame(train_y)
train_y_d.apply(pd.value_counts)

,0
16,174305
21,125943
20,91911
1,76811
7,53919
36,53697
35,44580
37,42137
4,36599
32,31392


In [22]:
test

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
Id,,,,,,,,,,,
0,6,0,-122.399588,37.735051,4512,10,5,2015,23,59,True
1,6,0,-122.391523,37.732432,4512,10,5,2015,23,51,False
2,6,4,-122.426002,37.792212,4512,10,5,2015,23,50,True
3,6,2,-122.437394,37.721412,4512,10,5,2015,23,45,True
4,6,2,-122.437394,37.721412,4512,10,5,2015,23,45,True
...,...,...,...,...,...,...,...,...,...,...,...
884257,2,3,-122.408983,37.751987,0,1,1,2003,0,1,True
884258,2,4,-122.425342,37.792681,0,1,1,2003,0,1,True
884259,2,2,-122.445418,37.712075,0,1,1,2003,0,1,True


## ENN

In [0]:
# enn_train_x, enn_train_y = EditedNearestNeighbours(kind_sel="mode", n_neighbors=5, random_state=0).fit_sample(train_x, train_y)

In [0]:
# enn_train_x

In [0]:
# enn_X_train=pd.DataFrame(enn_train_x)
# enn_X_train.columns=['DayOfWeek',	'PdDistrict',	'X',	'Y',	'n_days',	'Day',	'Month',	'Year',	'Hour','Minute',	'Block']
# enn_X_train

In [0]:
# enn_y_d=pd.DataFrame(enn_train_y)
# enn_y_d.apply(pd.value_counts)

In [0]:
# enn_y=pd.Series(enn_train_y)
# enn_y.unique()

In [0]:
# from sklearn.model_selection import train_test_split
# enn_X_train, enn_X_valid, enn_y_train, enn_y_valid = train_test_split(enn_X_train, enn_train_y, \
#                                                   test_size=0.2, random_state=42, stratify = enn_train_y) # 안 됨

# Tomek Link

In [46]:
tl=TomekLinks()
tl_train_x,tl_train_y = tl.fit_sample(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [47]:
tl_X_train=pd.DataFrame(tl_train_x)
tl_X_train.columns=['DayOfWeek',	'PdDistrict',	'X',	'Y',	'n_days',	'Day',	'Month',	'Year',	'Hour','Minute',	'Block']
tl_X_train

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
0,2.0,4.0,-122.425892,37.774599,4510.0,13.0,5.0,2015.0,23.0,53.0,0.0
1,2.0,4.0,-122.425892,37.774599,4510.0,13.0,5.0,2015.0,23.0,53.0,0.0
2,2.0,4.0,-122.424363,37.800414,4510.0,13.0,5.0,2015.0,23.0,33.0,0.0
3,2.0,4.0,-122.426995,37.800873,4510.0,13.0,5.0,2015.0,23.0,30.0,1.0
4,2.0,5.0,-122.438738,37.771541,4510.0,13.0,5.0,2015.0,23.0,30.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
666842,0.0,6.0,-122.472985,37.782552,0.0,6.0,1.0,2003.0,0.0,31.0,0.0
666843,0.0,0.0,-122.389769,37.730564,0.0,6.0,1.0,2003.0,0.0,20.0,1.0
666844,0.0,0.0,-122.389769,37.730564,0.0,6.0,1.0,2003.0,0.0,20.0,1.0
666845,0.0,2.0,-122.447364,37.731948,0.0,6.0,1.0,2003.0,0.0,1.0,1.0


In [48]:
tl_y_d=pd.DataFrame(tl_train_y)
tl_y_d.apply(pd.value_counts)

,0
16,126660
21,100866
20,62537
1,60265
7,49112
36,41874
37,36251
35,29301
4,23396
32,21576


In [0]:
## CV model
# %%time
# xgb=XGBClassifier()
# kfold = StratifiedKFold(n_splits=5)
# results = cross_val_score(xgb, tl_X_train, tl_train_y, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
from sklearn.model_selection import train_test_split
tl_X_train, tl_X_valid, tl_y_train, tl_y_valid = train_test_split(tl_X_train, tl_train_y, \
                                                  test_size=0.2, random_state=42, stratify = tl_train_y)

In [51]:
tl_X_valid=pd.DataFrame(tl_X_valid)
tl_X_valid.columns=['DayOfWeek',	'PdDistrict',	'X',	'Y',	'n_days',	'Day',	'Month',	'Year',	'Hour','Minute',	'Block']
tl_X_valid

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block
197312,2.0,7.0,-122.409893,37.780113,3278.0,28.0,12.0,2011.0,13.0,29.0,1.0
118680,4.0,8.0,-122.474506,37.722113,3784.0,17.0,5.0,2013.0,0.0,1.0,1.0
533057,3.0,5.0,-122.432311,37.768377,878.0,2.0,6.0,2005.0,17.0,30.0,1.0
616693,6.0,3.0,-122.407538,37.765783,314.0,16.0,11.0,2003.0,2.0,0.0,0.0
454763,1.0,2.0,-122.448702,37.739277,1450.0,26.0,12.0,2006.0,0.0,45.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
246408,3.0,9.0,-122.412193,37.781605,2929.0,13.0,1.0,2011.0,3.0,58.0,1.0
287634,2.0,8.0,-122.468612,37.729598,2648.0,7.0,4.0,2010.0,9.0,45.0,0.0
329399,1.0,3.0,-122.419561,37.752959,2346.0,9.0,6.0,2009.0,17.0,45.0,1.0
329297,3.0,1.0,-122.407327,37.790399,2348.0,11.0,6.0,2009.0,1.0,0.0,1.0


# XGBoost

In [32]:
# %%time
# xgb_c1 = XGBClassifier(n_estimators=100, n_jobs=-1, learning_rate=0.05, max_depth=10)
# xgb_c1.fit(tl_X_train,tl_y_train, early_stopping_rounds=10,eval_metric='mlogloss',eval_set=[(tl_X_train, tl_y_train), (tl_X_valid, tl_y_valid)])

CPU times: user 30min 13s, sys: 2.25 s, total: 30min 15s
Wall time: 15min 21s


In [0]:
# xgb_c1_pred=xgb_c1.predict_proba(test)

In [3]:
# test_id

In [38]:
# submission=pd.DataFrame(data=xgb_c1_pred)
# submission=pd.concat([test_id,submission],axis=1)
# submission.columns=sample_submission.columns
# submission

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,0.004204,0.494023,0.002337,0.002505,0.034507,0.002612,0.004882,0.032543,0.046247,0.002323,0.002303,0.002303,0.002657,0.005154,0.002252,0.004007,0.019012,0.002413,0.002300,0.019213,0.066270,0.120568,0.002419,0.002287,0.033570,0.002281,0.004741,0.002606,0.002411,0.002246,0.013295,0.005738,0.003714,0.011213,0.002430,0.024120,0.012293
1,0.002042,0.087769,0.001805,0.001855,0.002382,0.002908,0.002811,0.038704,0.037597,0.001794,0.001779,0.001779,0.001979,0.002931,0.001740,0.003095,0.011330,0.001860,0.001777,0.002168,0.010646,0.678381,0.001793,0.001885,0.023933,0.001762,0.003368,0.002003,0.001871,0.001735,0.008818,0.001749,0.001901,0.003244,0.002089,0.018614,0.026103
2,0.006808,0.348222,0.002386,0.002439,0.031907,0.002542,0.004757,0.019975,0.010878,0.002374,0.002354,0.002353,0.002704,0.007487,0.002302,0.002611,0.134988,0.002460,0.002351,0.027980,0.038458,0.259998,0.002419,0.002337,0.013391,0.002331,0.003375,0.002908,0.002457,0.002296,0.008020,0.002314,0.003891,0.016186,0.003132,0.009214,0.007394
3,0.004650,0.440356,0.002457,0.002525,0.016299,0.004030,0.005233,0.025896,0.037431,0.002442,0.002422,0.002421,0.002785,0.009250,0.002368,0.014177,0.155906,0.002532,0.002419,0.013580,0.042631,0.072002,0.002416,0.002404,0.056631,0.002399,0.004584,0.002730,0.002528,0.002362,0.006925,0.002381,0.002587,0.017926,0.005534,0.015523,0.009258
4,0.004650,0.440356,0.002457,0.002525,0.016299,0.004030,0.005233,0.025896,0.037431,0.002442,0.002422,0.002421,0.002785,0.009250,0.002368,0.014177,0.155906,0.002532,0.002419,0.013580,0.042631,0.072002,0.002416,0.002404,0.056631,0.002399,0.004584,0.002730,0.002528,0.002362,0.006925,0.002381,0.002587,0.017926,0.005534,0.015523,0.009258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,0.001941,0.229078,0.001883,0.001911,0.059594,0.005957,0.001938,0.012916,0.002804,0.032947,0.001836,0.001834,0.110420,0.070329,0.001796,0.002017,0.035954,0.001938,0.001845,0.013939,0.035736,0.235484,0.001932,0.001823,0.001994,0.001819,0.003275,0.074527,0.004160,0.001791,0.002508,0.001806,0.003864,0.003870,0.024746,0.005403,0.002382
884258,0.002575,0.194538,0.002777,0.002503,0.079039,0.003384,0.002553,0.013932,0.002619,0.004487,0.002416,0.002413,0.142201,0.055361,0.002362,0.002670,0.209227,0.002549,0.003196,0.005099,0.036840,0.070120,0.003345,0.002399,0.002622,0.002393,0.003870,0.017177,0.007834,0.002356,0.004144,0.002375,0.002530,0.006122,0.091304,0.002672,0.005996
884259,0.002120,0.231482,0.001968,0.002015,0.111185,0.003095,0.002223,0.008582,0.002825,0.004335,0.001935,0.004556,0.199249,0.055304,0.001893,0.002125,0.057264,0.002043,0.001944,0.012018,0.049367,0.121980,0.001937,0.001922,0.002101,0.001917,0.003439,0.008674,0.003609,0.001888,0.002758,0.001903,0.002141,0.006403,0.055846,0.016874,0.009082
884260,0.007024,0.169818,0.001972,0.002011,0.037516,0.003343,0.002040,0.018896,0.002343,0.004510,0.001932,0.001930,0.168545,0.051381,0.001890,0.002092,0.032727,0.002039,0.001941,0.008367,0.039498,0.318728,0.001934,0.001961,0.002098,0.001914,0.003446,0.011393,0.007053,0.001885,0.003116,0.003216,0.002574,0.004128,0.042236,0.005073,0.027432


In [0]:
# submission.to_csv('/content/drive/My Drive/Colab Notebooks/Crime/Crime_XGB.csv', index=False)

# LGB

In [52]:
%%time
lgb_c2=lgb.LGBMClassifier(boosting_type='gbdt',num_leaves=50,max_depth=10,n_estimators=200,learning_rate=0.005,objective='multiclass')
lgb_c2.fit(tl_X_train,tl_y_train,early_stopping_rounds=10,eval_metric='logloss',eval_set=[(tl_X_train, tl_y_train), (tl_X_valid, tl_y_valid)])

[1]	training's multi_logloss: 2.69257	training's multi_logloss: 2.69257	valid_1's multi_logloss: 2.69325	valid_1's multi_logloss: 2.69325
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 2.6889	training's multi_logloss: 2.6889	valid_1's multi_logloss: 2.69003	valid_1's multi_logloss: 2.69003
[3]	training's multi_logloss: 2.68552	training's multi_logloss: 2.68552	valid_1's multi_logloss: 2.68701	valid_1's multi_logloss: 2.68701
[4]	training's multi_logloss: 2.6823	training's multi_logloss: 2.6823	valid_1's multi_logloss: 2.68411	valid_1's multi_logloss: 2.68411
[5]	training's multi_logloss: 2.67925	training's multi_logloss: 2.67925	valid_1's multi_logloss: 2.68137	valid_1's multi_logloss: 2.68137
[6]	training's multi_logloss: 2.6763	training's multi_logloss: 2.6763	valid_1's multi_logloss: 2.6787	valid_1's multi_logloss: 2.6787
[7]	training's multi_logloss: 2.67341	training's multi_logloss: 2.67341	valid_1's multi_logloss: 2.67606	valid_1's mul

In [0]:
lgb_c2_pred=lgb_c2.predict_proba(test)

In [54]:
submission=pd.DataFrame(data=lgb_c2_pred)
submission=pd.concat([test_id,submission],axis=1)
submission.columns=sample_submission.columns
submission

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.003164,0.136492,0.000297,0.000567,0.031347,0.004581,0.003962,0.065901,0.005557,0.002457,0.000255,0.000505,0.008434,0.014689,0.000191,0.004665,0.134819,0.002173,0.001113,0.039194,0.099215,0.144897,0.000015,0.007484,0.001875,0.028405,0.002070,0.020563,0.013314,0.000138,0.007313,0.000527,0.039762,0.000006,0.007846,0.047979,0.044871,0.057059,0.016296
1,1,0.003569,0.079030,0.000243,0.002287,0.020554,0.004203,0.005904,0.084481,0.005099,0.000673,0.000216,0.000402,0.007308,0.013478,0.000100,0.002534,0.123698,0.001994,0.001003,0.019877,0.076604,0.285470,0.000014,0.006024,0.001721,0.030128,0.001246,0.012820,0.003716,0.000132,0.007979,0.000340,0.029240,0.000005,0.005415,0.034379,0.041170,0.065533,0.021413
2,2,0.001369,0.109627,0.000395,0.000352,0.055796,0.004227,0.003457,0.053228,0.007651,0.000833,0.000258,0.000443,0.008019,0.017263,0.000116,0.003399,0.220611,0.002153,0.001017,0.026346,0.094920,0.132756,0.000014,0.006039,0.001765,0.027260,0.001342,0.014945,0.005767,0.000108,0.006226,0.000562,0.031718,0.000006,0.009426,0.046790,0.041455,0.052139,0.010203
3,3,0.002088,0.130508,0.000297,0.002960,0.035044,0.004584,0.003729,0.067005,0.005560,0.000937,0.000255,0.000611,0.008439,0.014824,0.000129,0.004225,0.141582,0.002191,0.001081,0.036071,0.099273,0.136580,0.000015,0.006561,0.001876,0.035963,0.003584,0.020897,0.006075,0.000138,0.007414,0.000579,0.038919,0.000006,0.008777,0.058476,0.044087,0.055033,0.013628
4,4,0.002088,0.130508,0.000297,0.002960,0.035044,0.004584,0.003729,0.067005,0.005560,0.000937,0.000255,0.000611,0.008439,0.014824,0.000129,0.004225,0.141582,0.002191,0.001081,0.036071,0.099273,0.136580,0.000015,0.006561,0.001876,0.035963,0.003584,0.020897,0.006075,0.000138,0.007414,0.000579,0.038919,0.000006,0.008777,0.058476,0.044087,0.055033,0.013628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,0.001315,0.093537,0.004445,0.000333,0.031791,0.004344,0.002222,0.047597,0.004048,0.048400,0.000456,0.000902,0.045824,0.053136,0.000190,0.003212,0.126568,0.001752,0.000981,0.027943,0.081626,0.144591,0.000014,0.008545,0.001735,0.016295,0.004925,0.015919,0.013816,0.000267,0.004710,0.001803,0.037843,0.000005,0.006882,0.045571,0.068391,0.036423,0.011644
884258,884258,0.001238,0.067783,0.013229,0.000288,0.055468,0.004874,0.002084,0.043510,0.003796,0.017191,0.000461,0.001282,0.042822,0.054513,0.000145,0.002887,0.182295,0.001562,0.000897,0.024289,0.080312,0.129923,0.000013,0.006741,0.001627,0.015006,0.001237,0.014696,0.006910,0.000101,0.004417,0.000491,0.043542,0.000005,0.006842,0.042741,0.083871,0.032899,0.008012
884259,884259,0.001495,0.086988,0.005198,0.000324,0.035008,0.004310,0.002216,0.046276,0.004038,0.010315,0.002402,0.001120,0.042669,0.045499,0.000108,0.003363,0.131150,0.001736,0.000954,0.029390,0.081433,0.138895,0.000014,0.006468,0.001731,0.015960,0.001943,0.016701,0.028254,0.000188,0.004698,0.001798,0.037933,0.000005,0.006865,0.045520,0.110506,0.034991,0.011537
884260,884260,0.002280,0.092650,0.013830,0.000877,0.049560,0.004424,0.002262,0.047428,0.004122,0.009145,0.000417,0.000918,0.045354,0.043857,0.000217,0.003271,0.129729,0.001667,0.000997,0.028407,0.079986,0.147252,0.000014,0.006603,0.001767,0.016595,0.001880,0.017508,0.012892,0.000456,0.004797,0.001468,0.038445,0.000006,0.007009,0.046409,0.086407,0.036079,0.013014


In [0]:
submission.to_csv('/content/drive/My Drive/Colab Notebooks/Crime/Crime_LGB.csv', index=False)
# score : 2.46608